In [11]:
import json
import re

def clean_data(data):
    cleaned_data = []

    for university in data:
        cleaned_university = {
            "มหาวิทยาลัย": university["มหาวิทยาลัย"],
            "รายละเอียด": {}
        }

        for branch, programs in university["รายละเอียด"].items():
            # Remove leading numbers from branch names
            clean_branch = re.sub(r"^\d+\.\s*", "", branch)
            cleaned_university["รายละเอียด"][clean_branch] = []

            for program in programs:
                clean_program = {}
                for key, value in program.items():
                    if key == "":
                        continue
                    elif key == "ค่าใช้จ่าย":
                        # Extract only the first numeric value from ค่าใช้จ่าย
                        match = re.search(r"\d{1,3}(?:,\d{3})*(?:\.\d+)?", value.replace(",", ""))
                        clean_program[key] = int(match.group().replace(",", "")) if match else "ไม่ทราบ"
                    elif key.startswith("รอบ"):
                        # Extract only numeric values from acceptance numbers
                        if "ไม่เปิดรับสมัครในรอบนี้" in value:
                            clean_program[key] = 0
                        else:
                            match = re.search(r"\d+", value)
                            clean_program[key] = int(match.group()) if match else 0
                    else:
                        clean_program[key] = value

                # Remove empty values
                clean_program = {k: v for k, v in clean_program.items() if v != ""}
                cleaned_university["รายละเอียด"][clean_branch].append(clean_program)

        cleaned_data.append(cleaned_university)

    return cleaned_data

# Load JSON data
with open("../data/all_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

# Clean data
cleaned_data = clean_data(data)

# Save cleaned data
with open("../data/cleaned_universities_data.json", "w", encoding="utf-8") as file:
    json.dump(cleaned_data, file, ensure_ascii=False, indent=4)

print("Data cleaned and saved to cleaned_data.json")


Data cleaned and saved to cleaned_data.json


#Convert to CSV

In [1]:
import json
import csv

with open("../data/cleaned_universities_data.json", "r", encoding="utf-8") as file:
    data = json.load(file)

csv_data = []
csv_headers = [
    "มหาวิทยาลัย", "คณะ", "ชื่อหลักสูตร", "ชื่อหลักสูตรภาษาอังกฤษ", "ประเภทหลักสูตร", "วิทยาเขต",
    "ค่าใช้จ่าย", "อัตราการสำเร็จการศึกษา", "อัตราการได้งานทำ", "ค่ามัธยฐานเงินเดือน",
    "รอบ 1 Portfolio", "รอบ 2 Quota", "รอบ 3 Admission", "รอบ 4 Direct Admission"
]

for university in data:
    university_name = university["มหาวิทยาลัย"]
    for branch, programs in university["รายละเอียด"].items():
        for program in programs:
            row = {
                "มหาวิทยาลัย": university_name,
                "คณะ": branch,
                "ชื่อหลักสูตร": program.get("ชื่อหลักสูตร", ""),
                "ชื่อหลักสูตรภาษาอังกฤษ": program.get("ชื่อหลักสูตรภาษาอังกฤษ", ""),
                "ประเภทหลักสูตร": program.get("ประเภทหลักสูตร", ""),
                "วิทยาเขต": program.get("วิทยาเขต", ""),
                "ค่าใช้จ่าย": program.get("ค่าใช้จ่าย", ""),
                "อัตราการสำเร็จการศึกษา": program.get("อัตราการสำเร็จการศึกษา", ""),
                "อัตราการได้งานทำ": program.get("อัตราการได้งานทำ", ""),
                "ค่ามัธยฐานเงินเดือน": program.get("ค่ามัธยฐานเงินเดือน", ""),
                "รอบ 1 Portfolio": program.get("รอบ 1 Portfolio", ""),
                "รอบ 2 Quota": program.get("รอบ 2 Quota", ""),
                "รอบ 3 Admission": program.get("รอบ 3 Admission", ""),
                "รอบ 4 Direct Admission": program.get("รอบ 4 Direct Admission", "")
            }
            csv_data.append(row)

with open("../data/cleaned_universities_data.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.DictWriter(file, fieldnames=csv_headers)
    writer.writeheader()
    writer.writerows(csv_data)

print("Data has been converted to CSV and saved to data.csv")


Data has been converted to CSV and saved to data.csv
